# Mountain Car

#### Descripcion 


+ **Espacio de observacion (Box(2,))**

+ Posicion: [-1.2, 0.6]
+ Velocidad: [-0.07, 0.07]


+ **Espacio de accion (Discrete(3))**

+ Izquierda: 0
+ No hacer nada: 1
+ Derecha: 2


El objetivo es llevar el coche hasta la posicion 0.5

Para cada paso temporal que el coche no este en esa posicion, la recompensa sera -1.

In [1]:
# primero test aleatorio del entorno

import gym

In [2]:
entorno=gym.make('MountainCar-v0')

In [4]:
for episodio in range(10):  # 10 episodios
    
    done=False
    
    observacion=entorno.reset()  # observacion
    
    recompensa_total=0.          # recompensa total en cada episodio
    
    paso=0                       # paso en cada episodio
     
    while not done:
        entorno.render()  # muestra el entorno
        
        accion=entorno.action_space.sample() # accion aleatoria, se cambiara por el agente
        
        siguiente_estado, recompensa, done, info=entorno.step(accion)
        
        recompensa_total+=recompensa
        
        paso+=1
        
        observacion=siguiente_estado
        
    print ('\nEpisodio #{} finalizado en {} pasos. Recompensa total:{}'.format(episodio+1, paso, recompensa_total))
    
entorno.close()


Episodio #1 finalizado en 200 pasos. Recompensa total:-200.0

Episodio #2 finalizado en 200 pasos. Recompensa total:-200.0

Episodio #3 finalizado en 200 pasos. Recompensa total:-200.0

Episodio #4 finalizado en 200 pasos. Recompensa total:-200.0

Episodio #5 finalizado en 200 pasos. Recompensa total:-200.0

Episodio #6 finalizado en 200 pasos. Recompensa total:-200.0

Episodio #7 finalizado en 200 pasos. Recompensa total:-200.0

Episodio #8 finalizado en 200 pasos. Recompensa total:-200.0

Episodio #9 finalizado en 200 pasos. Recompensa total:-200.0

Episodio #10 finalizado en 200 pasos. Recompensa total:-200.0


### Implementacion del agente

Ahora vamos a crear un agente basado en el algoritmo Q-learning (Q viene de quality, la calidad de la accion tomada en un estado dado). Dicho algoritmo tiene la ecuacion:

$$Q_{nueva}(e_{t}, a_{t})=(1-\alpha)·Q(e_{t}, a_{t}) + \alpha·[r_{t} + \gamma·\max_{a}Q(e_{t+1}, a_{t+1})]$$


donde:
+ $e_{t}$ es el estado en el tiempo t
+ $a_{t}$ es la accion en el tiempo t
+ $\alpha$ es la tasa de aprendizaje $(0<\alpha \leq{1})$
+ $Q(e_{t}, a_{t})$ es el viejo valor de calidad
+ $[r_{t} + \gamma·\max_{a}Q(e_{t+1}, a_{t+1})]$ es el valor aprendido
+ $r_{t}$ es la recompensa recibida al pasar del estado $e_{t}$ al estado $e_{t+1}$
+ $\gamma$ es el factor de descuento $(0\leq \gamma \leq 1)$. Evalua las recompensas recibidas anteriormente con un valor mayor que las recibidas posteriormente, se puede interpretar como la probabilidad de tener exito (o sobrevivir) en cada paso temporal
+ $\max_{a}Q(e_{t+1}, a_{t+1})$ es la estimacion del vaalor optimo futuro

In [6]:
# se crea la clase agente (basada en numpy)

import numpy as np

class Agente(object):
    
    def __init__(self, entorno):
        self.obs_dim=entorno.observation_space.shape              # dimension espacio observacion
        self.obs_sup=entorno.observation_space.high               # limite superior
        self.obs_inf=entorno.observation_space.low                # limite inferior
        self.obs_bins=NUM_BINS                                    # discretizacion de un espacio continuo, numero de bins
        self.ancho_bin=(self.obs_sup-self.obs_inf)/self.obs_bins  # ancho de cada parte de la discretizacion
        self.dim_accion=entorno.action_space.n                    # dimension espacio accion
        self.Q=np.zeros((self.obs_bins+1, 
                         self.obs_bins+1,
                         self.dim_accion))  # array para guardar los Q-valores
        
        self.alfa=ALFA        # tasa de aprendizaje           
        self.gamma=GAMMA      # factor de descuento
        self.epsilon=EPSILON  # 
        
        
    def discretiza(self, obs):  # discretiza un espacio continuo
        return tuple(((self.obs_sup-self.obs_inf)/self.obs_bins).astype(int))  # binning
        
        
    def accion(self, obs):   # realiza la accion del agente
        obs_discreta=self.discretiza(obs) # discretiza espacio
        
        # politica epsilon-greedy para escoger la accion (mejor probabilidad 1-eps, peor es epsilon)
        if self.epsilon>MIN_EPSILON: self.epsilon-=DECAY_EPSILON
            
        if np.random.random()>self.epsilon: return np.argmax(self.Q[obs_discreta])
        else: return np.random.choice([a for a in range(self.dim_accion)])
        
        
    def aprende(self, obs, acciona, recompensa, siguiente_obs): # metodo de aprendizaje
        obs_discreta=self.discretiza(obs)    # discretiza las observaciones
        siguiente_obs_discreta=self.discretiza(siguiente_obs)
        
        td_objetivo=recompensa+self.gamma*np.argmax(self.Q[siguiente_obs_discreta]) # valor de aprendizaje
        td_error=td_objetivo-self.Q[obs_discreta][acciona]   # error aprendizaje
        self.Q[obs_discreta][acciona]+=self.alfa*td_error    # actualizacion Q-valores